<a href="https://colab.research.google.com/github/mingmingbupt/tensorflow/blob/master/tf1_dense_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%tensorflow_version 1.x
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)


TensorFlow is already loaded. Please restart the runtime to change versions.
1.15.2
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.2
pandas 1.0.3
sklearn 0.22.2.post1
tensorflow 1.15.2
tensorflow.python.keras.api._v1.keras 2.2.4-tf


In [9]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [10]:
print(np.max(x_train), np.min(x_train))

255 0


In [0]:
# x = (x - u) / std

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# x_train: [None, 28, 28] -> [None, 784]
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28 * 28)
x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28 * 28)
x_test_scaled = scaler.transform(
    x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28 * 28)


In [12]:
print(np.max(x_train_scaled), np.min(x_train_scaled))

2.0231433 -0.8105136


In [6]:
hidden_units = [100, 100] # 2层 每层100的单元的全连接网络
class_num = 10 #10类

x = tf.placeholder(tf.float32, [None, 28 * 28]) #x用来存放数据也就是图像。placeholder也就占位符，占位符是网络中的一部分
#数据通过占位符输入进网络，网络再通过占位符呢读取到数据之后呢，他可能会做两件事情
#第一件就是： 给这些数据做预测
#第二件就是： 利用这些数据去做训练
#x 用来存放图像，28*28是图像的大小，None是batch_size的大小。表示batch_size可以是任意值
#y shape也是none,他需要跟x做对应。
#placeholder呢在2.0里呢已经被取消了，这是为什么呢
#在tensorflow1.0里面呢，需要先构建图，再去运行这个图。所以在构建图的过程中是看不到数据的。
#所以无法把数据传进来，所以需要一个占位符顶替数据的位置。在tensorflow2.0里面则不需要这样了

y = tf.placeholder(tf.int64, [None]) #y用来存放label。

#定义一个临时变量
input_for_next_layer = x
for hidden_unit in hidden_units:
    #构建一个新的层次，在tensorflow1.0里面构建全连接层的API是tf.layers.dense，他在2.0已经被移到keras.layers.dense里面去了
    #参数有三个： 第一个输入，hidden_unit数目，激活函数
    input_for_next_layer = tf.layers.dense(input_for_next_layer,
                                           hidden_unit,
                                           activation=tf.nn.relu)
#输出层 也是一个全连接层，输入就是input_for_next_layer， unit数目就是class_num，他没有激活函数
logits = tf.layers.dense(input_for_next_layer, class_num)
# last_hidden_output * W(logits) -> softmax -> prob
# 1. logit -> softmax -> prob
# 2. labels -> one_hot
# 3. calculate cross entropy
# 有了yhat以后就可以和y去计算损失函数了.这里使用softmax_cross_entropy来计算，他的API就是 tf.losses.sparse_softmax_cross_entropy
# 他有两个参数，logits就是最后一层隐含层乘以一个矩阵得到的一个直接的输出这个就是logits。 这个logits输出再经过一个softmax以后呢，就变成了概率。
# 也就是在概率输出前的叫logits
# 在这个函数中呢，他会首先给logits计算概率
# 还会给labels计算one_hot编码，有了one-hot编码和概率以后，他就可以去计算交叉熵cross entropy
# 这是这个函数所做的事情，
loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = logits)
# get accuracy.
prediction = (tf.argmaxlogits, 1) #他的proiction就是logits最大的那个值的索引 https://blog.csdn.net/UESTC_C2_403/article/details/72232807
correct_prediction = tf.equal(prediction, y) #01的一个向量，0是错误，1是正确
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64)) #取平均就好，取平均前，变成float64,不然最后就是0

train_op = tf.train.AdamOptimizer(1e-3).minimize(loss) #train_op用来训练网络的，这个train_op运行一次，网络就训练一次
#tf.train.AdamOptimizer(1e-3).minimize(loss)返回一个train_op. train_op也是tf1.0的一个特性，在tf1.0里面要先构建图
#然后在去训练，train_op也是为了这个逻辑而存在的，在构建完图以后呢，train_op也被构建为图的一部分，我只要运行这个train_op
#就相当于把这个图训练了一遍

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
print(x)
print(logits)

Tensor("Placeholder:0", shape=(?, 784), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(?, 10), dtype=float32)


In [14]:
# session
# 接下来运行这个图
# session是客户端程序和c++ 运行时的一个连接
# 什么是c++运行时，我们在构建好图以后，图就是一个c++运行时，因为tensorflow的图是c++实现的
# 客户端一般指的是我们的python程序
# session就是这些客户端和c++运行时的一个连接
# 打开一个session以后，就可以在客户端里运行c++运行时了
init = tf.global_variables_initializer()
batch_size = 20
epochs = 10
train_steps_per_epoch = x_train.shape[0] // batch_size
valid_steps = x_valid.shape[0] // batch_size

#验证集的返回结果
def eval_with_sess(sess,  accuracy, images, labels, batch_size):
    eval_steps = images.shape[0] // batch_size
    eval_accuracies = []
    for step in range(eval_steps):
        batch_data = images[step * batch_size : (step+1) * batch_size]
        batch_label = labels[step * batch_size : (step+1) * batch_size]
        accuracy_val = sess.run(accuracy, #我不去调用train_op,我只需要accuracy就可以了
                                feed_dict = {
                                    x: batch_data,
                                    y: batch_label
                                })
        eval_accuracies.append(accuracy_val)
    return np.mean(eval_accuracies)

#很类似keras的fit函数，遍历训练集，然后在测试集验证下
with tf.Session() as sess: #打开session
    sess.run(init) #运行初始化
    for epoch in range(epochs):
        for step in range(train_steps_per_epoch):
            batch_data = x_train_scaled[ #取出来对应的data
                step * batch_size : (step+1) * batch_size]
            batch_label = y_train[  #取出来label
                step * batch_size : (step+1) * batch_size]
            #不关心train_op的值，只是关心loss_val和accuracy_val
            loss_val, accuracy_val, _ = sess.run( #使用sess.run调用train_op
                [loss, accuracy, train_op],
                feed_dict = {
                    x: batch_data,
                    y: batch_label
                })
            print('\r[Train] epoch: %d, step: %d, loss: %3.5f, accuracy: %2.2f' % (
                epoch, step, loss_val, accuracy_val), end="") #打印中间训练过程
        #每个epoch结束以后呢，需要在验证集上进行一下验证
        valid_accuracy = eval_with_sess(sess,  accuracy,
                                        x_valid_scaled, y_valid,
                                        batch_size)
        print("\t[Valid] acc: %2.2f" % (valid_accuracy))

[Train] epoch: 0, step: 2749, loss: 0.19854, accuracy: 0.90	[Valid] acc: 0.86
[Train] epoch: 1, step: 2749, loss: 0.17359, accuracy: 0.90	[Valid] acc: 0.87
[Train] epoch: 2, step: 2749, loss: 0.12006, accuracy: 0.95	[Valid] acc: 0.88
[Train] epoch: 3, step: 2749, loss: 0.14087, accuracy: 0.90	[Valid] acc: 0.88
[Train] epoch: 4, step: 2749, loss: 0.19757, accuracy: 0.90	[Valid] acc: 0.88
[Train] epoch: 5, step: 2749, loss: 0.19841, accuracy: 0.90	[Valid] acc: 0.88
[Train] epoch: 6, step: 2749, loss: 0.14377, accuracy: 0.90	[Valid] acc: 0.88
[Train] epoch: 7, step: 2749, loss: 0.15756, accuracy: 0.90	[Valid] acc: 0.88
[Train] epoch: 8, step: 2749, loss: 0.11145, accuracy: 0.95	[Valid] acc: 0.87
[Train] epoch: 9, step: 2749, loss: 0.12014, accuracy: 0.95	[Valid] acc: 0.87
